<h1> On Gradients with Effective-Quadratures

In [1]:
%matplotlib inline
from effective_quadratures.parameter import Parameter
from effective_quadratures.polynomial import Polynomial
from effective_quadratures.indexset import IndexSet
from effective_quadratures.computestats import Statistics
from effective_quadratures.effectivequads import EffectiveSubsampling
from effective_quadratures.utils import meshgrid, twoDgrid, evalfunction, evalgradients
import numpy as np
import matplotlib.pyplot as plt
import math
np.set_printoptions(precision=3)

Consider the following function and its gradient:
$$f(x_1, x_2) = exp( x_1 + x_2),$$
$$\nabla f=\left(\begin{array}{c}
exp( x_1 + x_2) + \gamma\\
exp( x_1 + x_2) + \gamma
\end{array}\right).$$

We start by defining our computational model:

In [2]:
def fun(x):
    return exp(x[0] + x[1])

def fungrad(x):
    return [exp(x[0] + x[1]) + 0.1*np.random.rand(1), exp(x[0] + x[1]) + 0.1*np.random.rand(1)] 

def fungrad_noiseless(x):
    return [exp(x[0] + x[1]), exp(x[0] + x[1])] 

In [3]:
value = 6
x1 = Parameter(param_type="Uniform", lower=-1, upper=1, points=value, derivative_flag=1)
x2 = Parameter(param_type="Uniform", lower=-1, upper=1, points=value, derivative_flag=1)
parameters = [x1, x2]

Now select a hyperbolic basis

In [4]:
hyperbolic_cross = IndexSet("Hyperbolic basis", orders=[value-1,value-1], q=1.0)
maximum_number_of_evals = hyperbolic_cross.getCardinality()

Some useful text goes here...

In [5]:
esq = EffectiveSubsampling(parameters, hyperbolic_cross)
terms = esq.no_of_basis_terms

Now we set the number of function evaluations to be equivalent to the number of basis terms. 

In [6]:
esq.set_no_of_evals(terms)
x =  esq.computeCoefficients(fun)

In [7]:
evals = int(terms/2)
print evals
esq.set_no_of_evals(evals)
x1 =  esq.computeCoefficients(fun, fungrad,'weighted')
x1b =  esq.computeCoefficients(fun, fungrad,'equality')
x2 =  esq.computeCoefficients(fun, fungrad_noiseless,'weighted')
weighted_error = np.linalg.norm(x-x1, 2)
equality_error = np.linalg.norm(x-x1b, 2)
weighted_error_noiseless = np.linalg.norm(x-x2, 2)

10


In [8]:
print weighted_error, equality_error, weighted_error_noiseless

0.0550700301519 0.0497520212556 1.03492000795e-14


Now start pruning down the columns

In [9]:
esq.set_no_of_evals(evals)
cols_to_prune = 3
esq.prune(cols_to_prune)
x1 =  esq.computeCoefficients(fun, fungrad,'weighted')
x1b =  esq.computeCoefficients(fun, fungrad_noiseless,'equality')
x2 =  esq.computeCoefficients(fun, fungrad_noiseless,'weighted')
xnew =  x[0:(terms-cols_to_prune)]
weighted_error = np.linalg.norm(xnew-x1, 2)
equality_error = np.linalg.norm(xnew-x1b, 2)
weighted_error_noiseless = np.linalg.norm(xnew-x2, 2)
print weighted_error, equality_error, weighted_error_noiseless

0.0420764503631 1.63883844104e-14 2.44558458464e-14
